In [ ]:
from IPython.display import display, Markdown, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import math
import ROOT
import numpy as np, pandas as pd
import yaml
import base64
import ctypes
import logging
import bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.Tools.compressArray import arrayCompressionRelative8

output_notebook()

# Display skimmed AO2D tracks

In [ ]:
#inputFile="/lustre/alice/users/miranov/NOTESData/alice-tpc-notes/JIRA/ATO-592/tpcqcskimmingTracks.root"
inputFile="~/Software/TPCQCVis/data/tpcqcskimmingTracks.root"

In [ ]:
f=ROOT.TFile.Open(inputFile)
tracks = f.Get("tracks;2")  
tracks.SetAlias("weight1","((triggerMask&0x1)>0)*max(weight,0.01)")
tracks.SetAlias("rMin","trackPar.mX")
tracks.SetAlias("qPt","trackPar.mP[4]")
tracks.SetAlias("pzPt","trackPar.mP[3]")
tracks.SetAlias("itsOn","itsClusterMap>0")

tracks.SetAlias("tofOn","abs(tofChi2)<100")

In [ ]:
df=tree2Panda(tracks, [".*"], "", exclude=["trackPar"])   
df["side"]=df.eval("pzPt>0") ##  (A/C/CrossAC/CrossCA side)
df["isPrim5"]=df.eval("abs(dcaXY)<5 &  abs(dcaZ)<5 & rMin<5")
df["tpcNCR"]=df["tpcNClsFindable"]-df["tpcNClsFindableMinusCrossedRows"]
df.head()

In [ ]:
aliasArray = [
    {
        "name": "logPtWeight",
        "variables": ["bin_count"],
        "func": "return Math.log(bin_count)",
        "context": "ptWeightedHist"
    }
]

In [ ]:
parameterArray = [
    {"name": "size", "value":5, "range":[0, 20]},
    {"name": "legendFontSize", "value":"13px", "options":["9px", "11px", "13px", "15px"]},
    {"name": "binCountTrans", "value":"bin_count", "options":["bin_count", "logbin_count","sqrtbin_count"]},
    {"name": "legendVisible", "value":True},
    {"name": "legendLocation", "value":"top_right", "options":["top_right","top_left", "bottom_right","bottom_left"]},
]

In [ ]:
widgetParams=[
      ['range', ['pt']],
      ['range', ['qPt']],
      ['range', ['phi']],
      ['range', ['pzPt']],
      #
      ['range', ['dcaXY']],
      ['range', ['dcaZ']],
      ['range', ['rMin']],
      ['range', ['tpcNClsFindable']],
      #
      ['multiSelect',["side"]],
      ['multiSelect',["isPrim5"]],
      ['multiSelect',["itsOn"]],
      ['multiSelect',["tofOn"]],
      ['multiSelect',["hasCollision"]],
      ['textQuery', {"title": "user select"}],
      # Graphics
      ['slider',["size"], {"callback": "parameter", "title":"Marker size"}],
      ['select',["binCountTrans"], {"callback": "parameter", "default": 1, "title":"Bin count transformation"}],
      ['toggle',['legendVisible']],  
      ['select',["legendFontSize"], {"callback": "parameter", "default": 2, "title":"Legend font size"}],
      ['select',["legendLocation"], {"callback": "parameter", "default": 0}]
]
widgetLayoutDesc={
    "Selection:": [[0,1,2,3],[4,5,6,7],[8,9,10,11,12],[13], {'sizing_mode': 'scale_width'}],
    "Graphics": [[14,15,16,17,18], {'sizing_mode': 'scale_width'}],
}

In [ ]:
histoArray = [
    #{"name": "ptHist", "variables": ["pt"], "nbins":200, "range": [0, 20]},
    #{"name": "phiHist", "variables": ["phi"], "nbins":180, "range": [0, 6.28]},
    #{"name": "phiHistA", "variables": ["phi"], "nbins":90},
    #{"name": "ptWeightedHist", "variables": ["pt"], "nbins":200, "range": [0, 20], "weights":"weight"},
    #{"name": "weightHist", "variables": ["weight"], "nbins":200, "range": [0, 20]},
    #{"name": "ptWeightHist", "variables": ["pt","weight"], "nbins":[200,200], "range": [[0, 20],[0,20]]},
    #{"name": "phiWeightedHist", "variables": ["phi"], "nbins":180, "range": [0, 6.28], "weights":"weight"},
    {"name": "ptVsPhi", "variables": ["phi","pt"], "nbins":[90,40], "range": [[0, 6.28],[0,20]]},
    {"name": "ptVstpcNClsFindable", "variables": ["tpcNClsFindable","pt"], "nbins":[80,40], "range": [[0,160],[0,20]]},
    {"name": "tpcSignalVspt", "variables": ["pt","tpcSignal"], "nbins":[80,80], "range": [[0,20],[0, 400]]},
    {"name": "tpcSignalVsPhi", "variables": ["phi","tpcSignal"], "nbins":[90,80], "range": [[0, 6.28],[0, 400]]},
]

histArrayPhi=[
    {"name": "dcaXYVsPhi", "variables": ["phi","dcaXY"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "dcaZVsPhi", "variables": ["phi","dcaZ"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcNClsFindableVsPhi", "variables": ["phi","tpcNClsFindable"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcNCRVsPhi", "variables": ["phi","tpcNCR"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcChi2NClVsPhi", "variables": ["phi","tpcChi2NCl"], "nbins":[30,20], "range": [None,[0, 6.28]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
]
histoArray.extend(histArrayPhi)

histArrayPt=[
    {"name": "dcaXYVsPt", "variables": ["pt","dcaXY"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "dcaZVsPt", "variables": ["pt","dcaZ"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcNClsFindableVsPt", "variables": ["pt","tpcNClsFindable"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcNCRVsPt", "variables": ["pt","tpcNCR"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "tpcChi2NClVsPt", "variables": ["pt","tpcChi2NCl"], "nbins":[30,20], "range": [None,[0, 6.28]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
]
histoArray.extend(histArrayPt)

histArraytpcNCR=[
    {"name": "dcaXYVstpcNCR", "variables": ["tpcNCR","dcaXY"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    {"name": "dcaZVstpcNCR", "variables": ["tpcNCR","dcaZ"], "nbins":[30,80], "range": [None, [-6,6]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    #{"name": "tpcNClsFindableVsPt", "variables": ["pt","tpcNClsFindable"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    #{"name": "tpcNCRVsPt", "variables": ["pt","tpcNCR"], "nbins":[30,20], "range": [None,[0, 159]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
    #{"name": "tpcChi2NClVsPt", "variables": ["pt","tpcChi2NCl"], "nbins":[30,20], "range": [None,[0, 6.28]], "quantiles": [.05, .5, .95],"axis": [0, 1]},
]
histoArray.extend(histArraytpcNCR)

In [ ]:
histoArrayLog=[]
for his in histoArray:
    histoName=his["name"]    
    histoArrayLog.append({ "name": f"log{histoName}","variables": ["bin_count"],"func": "return Math.log(bin_count+0.2)","context": f"{histoName}"})
    histoArrayLog.append({ "name": f"log{histoName}Err","variables": ["bin_count"],"func": "return 1/Math.sqrt(bin_count+0.2)","context": f"{histoName}"})               
    histoArrayLog.append({ "name": f"{histoName}Err","variables": ["entries","std"],"func": "return std/Math.sqrt(entries+0.1)","context": f"{histoName}_1"})
    #
    histoArrayLog.append({ "name": f"logbin_count","variables": ["bin_count"],"func": "return Math.log(bin_count+0.2)","context": f"{histoName}"})
    histoArrayLog.append({ "name": f"sqrtbin_count","variables": ["bin_count"],"func": "return Math.sqrt(bin_count+0.2)","context": f"{histoName}"})
aliasArray.extend(histoArrayLog)

In [ ]:
tooltips = []

In [ ]:
figureArray = [
    # Phi DCA
    [["bin_center_1"],["logdcaXYVsPhi"],{"source": "dcaXYVsPhi","colorZvar":"bin_center_0","errY":"logdcaXYVsPhiErr",
                                         "xAxisTitle":"DCA XY","yAxisTitle":"log(counts)"}],
    [["bin_center_1"],["logdcaZVsPhi"],{"source": "dcaZVsPhi","colorZvar":"bin_center_0","errY":"logdcaZVsPhiErr",
                                        "xAxisTitle":"DCA Z","yAxisTitle":"log(counts)"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaXYVsPhi_1","errY":"dcaXYVsPhiErr",
                                      "xAxisTitle":"Phi","yAxisTitle":"dcaXY (Quantile 0.5)"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaZVsPhi_1","errY":"dcaZVsPhiErr",
                                      "xAxisTitle":"Phi","yAxisTitle":"dcaZ (Quantile 0.5)"}],
    # Phi
    [["bin_center_1"],["logtpcNClsFindableVsPhi"],{"source": "tpcNClsFindableVsPhi","colorZvar":"bin_center_0","errY":"logtpcNClsFindableVsPhiErr",
                                                   "xAxisTitle":"NClsFindable"}],
    [["bin_center_1"],["logtpcNCRVsPhi"],{"source": "tpcNCRVsPhi","colorZvar":"bin_center_0","errY":"logtpcNCRVsPhiErr",
                                          "xAxisTitle":"TPC NCR"}],
    [["bin_center_1"],["logtpcChi2NClVsPhi"],{"source": "tpcChi2NClVsPhi","colorZvar":"bin_center_0","errY":"logtpcChi2NClVsPhiErr"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcNClsFindableVsPhi_1",
                                      "xAxisTitle":"Phi","yAxisTitle":"tpcNClsFindable (Quantile 0.5)"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcNCRVsPhi_1",
                                      "xAxisTitle":"Phi","yAxisTitle":"tpcNCR (Quantile 0.5)"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcChi2NClVsPhi_1",
                                      "xAxisTitle":"Phi","yAxisTitle":"tpcChi2 (Quantile 0.5)"}],
    # Pt DCA
    [["bin_center_1"],["logdcaXYVsPt"],{"source": "dcaXYVsPt","colorZvar":"bin_center_0","errY":"logdcaXYVsPtErr",
                                        "xAxisTitle":"DCA XY","yAxisTitle":"log(counts)"}],
    [["bin_center_1"],["logdcaZVsPt"],{"source": "dcaZVsPt","colorZvar":"bin_center_0","errY":"logdcaZVsPtErr",
                                       "xAxisTitle":"DCA Z","yAxisTitle":"log(counts)"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaXYVsPt_1","errY":"dcaXYVsPtErr",
                                      "xAxisTitle":"pT","yAxisTitle":"dcaXY (Quantile 0.5)"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaZVsPt_1","errY":"dcaZVsPtErr",
                                      "xAxisTitle":"pT","yAxisTitle":"dcaZ (Quantile 0.5)"}],
    # Pt
    [["bin_center_1"],["logtpcNClsFindableVsPt"],{"source": "tpcNClsFindableVsPt","colorZvar":"bin_center_0","errY":"logtpcNClsFindableVsPtErr",
                                                  "xAxisTitle":"NClsFindable"}],
    [["bin_center_1"],["logtpcNCRVsPt"],{"source": "tpcNCRVsPt","colorZvar":"bin_center_0","errY":"logtpcNCRVsPtErr",
                                         "xAxisTitle":"TPC NCR"}],
    [["bin_center_1"],["logtpcChi2NClVsPt"],{"source": "tpcChi2NClVsPt","colorZvar":"bin_center_0","errY":"logtpcChi2NClVsPtErr"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcNClsFindableVsPt_1",
                                      "xAxisTitle":"pT","yAxisTitle":"tpcNClsFindable (Quantile 0.5)"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcNCRVsPt_1",
                                      "xAxisTitle":"pT","yAxisTitle":"tpcNCR (Quantile 0.5)"}],
    [["bin_center_0"],["quantile_1"],{"source": "tpcChi2NClVsPt_1",
                                      "xAxisTitle":"pT","yAxisTitle":"tpcChi2 (Quantile 0.5)"}],
    # tpcNCR DCA
    [["bin_center_1"],["logdcaXYVstpcNCR"],{"source": "dcaXYVstpcNCR","colorZvar":"bin_center_0","errY":"logdcaXYVstpcNCRErr",
                                            "xAxisTitle":"DCA XY","yAxisTitle":"log(counts)"}],
    #[["bin_center_1"],["binCountTrans"],{"source": "dcaXYVstpcNCR","colorZvar":"bin_center_0","errY":"logdcaXYVstpcNCRErr","xAxisTitle":"DCA XY"}],
    [["bin_center_1"],["logdcaZVstpcNCR"],{"source": "dcaZVstpcNCR","colorZvar":"bin_center_0","errY":"logdcaZVstpcNCRErr",
                                           "xAxisTitle":"DCA Z","yAxisTitle":"log(counts)"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaXYVstpcNCR_1","errY":"dcaXYVstpcNCRErr",
                                      "xAxisTitle":"NCR","yAxisTitle":"dcaXY (Quantile 0.5)"}],
    [["bin_center_0"],["quantile_1"],{"source": "dcaZVstpcNCR_1","errY":"dcaZVstpcNCRErr",
                                      "xAxisTitle":"NCR","yAxisTitle":"dcaZ (Quantile 0.5)"}],
    # Overview
    [['pt'],['tpcSignalVsPhi'],{"xAxisTitle":"phi", "yAxisTitle":"tpcSignal"}],
    [['pt'],['tpcSignalVspt'],{"xAxisTitle":"pT", "yAxisTitle":"tpcSignal"}],
    [['pt'],['ptVsPhi'],{"xAxisTitle":"phi", "yAxisTitle":"pT"}],
    [['pt'],['ptVstpcNClsFindable'],{"xAxisTitle":"NClusters findable", "yAxisTitle":"pT"}],
    #[['pt'],['ptHist'],{"xAxisTitle":"pT", "yAxisTitle":"Counts"}],
    #[['bin_center'],['logbin_count'],{"source":"ptHist","xAxisTitle":"pT", "yAxisTitle":"log(Counts)"}],
    #[['pt'],['phiHistA'],{"xAxisTitle":"phi", "yAxisTitle":"Counts"}],
    
    {'plot_height':150, "commonX":0, "size":"size",
     "legend_options": {"label_text_font_size": "legendFontSize", "visible": "legendVisible", "location":"legendLocation"}}
]            

In [ ]:
figureLayout = {
    "Overview": {
        "TPC Signal": [[24,25], {'plot_height':400,'sizing_mode':'scale_width',"legend_visible":True}],
        "pT": [[26,27], {'plot_height':400,'sizing_mode':'scale_width',"legend_visible":True}]
    },
    "Phi": {
        "DCA phi":[[0,1],[2,3], {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}],
        "TPC N/Chi2, phi":[[4,5,6],[7,8,9], {'plot_height':300,'sizing_mode':'scale_width',"legend_visible":True}]
    },
    "Pt": {
        "DCA pt":[[10,11],[12,13], {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}],
        "TPC N/Chi2, pt":[[14,15,16],[17,18,19], {'plot_height':300,'sizing_mode':'scale_width',"legend_visible":True}]
    },
    "tpcNCR": {
        "DCA tpcNCR":[[20,21],[22,23], {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}]
    }
}

In [ ]:
output_file("investigating_skimmed_tracks.html")
xxx = bokehDrawSA.fromArray(df, "pt<20&abs(qPt)<20&abs(pzPt)<2", figureArray, widgetParams, layout=figureLayout, tooltips=tooltips, widgetLayout=widgetLayoutDesc,
                            parameterArray=parameterArray, histogramArray=histoArray,aliasArray=aliasArray,sizing_mode='scale_width',
                            arrayCompression=arrayCompressionRelative8)